In [1]:
import sys

sys.path.append("../")

In [2]:
from generators.instruction_generator import InstructionGenerator

generator = InstructionGenerator("../../models/Mistral-7B-Instruct-v0.2-q4")

In [3]:
from common.passage import Passage


query = "W jakim sposób eksploatowano model T.7A?"

context = "Z 55 zamówionych Hunterów T.7 45 trafiło do RAF-u, a pozostałych 10 do Royal Navy. Ponadto RAF dostał sześć maszyn przebudowanych z myśliwskich Hunterów F.4. Pięć maszyn zmodyfikowano przez dodanie systemu IFIS (OR946 Integrated Flight Instrumentation System) stosowanego na pokładach samolotów English Electric Lightning i Blackburn Buccaneer. Samoloty te zyskały nowe oznaczenie Hunter T.7A i były wykorzystywane do szkolenia załóg samolotów. Kolejna wersja Huntera oznaczona jako Hunter T.8 została pozbawiona części awioniki, ale wyposażona w hak do lądowania na lotniskowcach. Maszyny te nie były używane na morzu, a jedynie do ćwiczeń lądowania na lądzie. W 1957 opracowano kolejną wersję samolotu napędzaną nowszym i mocniejszym silnikiem Avon serii 200. Większa moc jednostki napędowej pozwoliła na zainstalowanie dwóch działek ADEN, ale tak wyposażona maszyna oznaczona jako Hunter T.66 nigdy nie weszła do służby w brytyjskich siłach zbrojnych. Jedna z maszyn tego typu oznaczona jako Hunter T.12 była testowana z awioniką ówcześnie projektowanego samolotu BAC TSR.2. Po zamknięciu projektu TSR.2 służyła ona w Royal Aircraft Establishment w Farnborough jako samolot testowy, między innymi do prób systemu fly by wire.	"

passage = Passage('', '', context, 0, '', '')


response = generator.generate_answer(query, [passage])

print(response)

Model Hunter T.7A był wykorzystywany do szkolenia załóg samolotów w Royal Air Force po przebudowie z dodaniem systemu IFIS.


In [3]:
from dataset.polqa_dataset_getter import PolqaDatasetGetter
from dataset.poquad_dataset_getter import PoquadDatasetGetter

poquad_dataset_getter = PoquadDatasetGetter()
polqa_dataset_getter = PolqaDatasetGetter()

poquad_dataset = poquad_dataset_getter.get_training_dataset()
polqa_dataset = polqa_dataset_getter.get_training_dataset()

In [15]:
import random
from common.dataset_entry import DatasetEntry


def get_prompt_for_training_dataset(entry: DatasetEntry, poquad_dataset: list[DatasetEntry], polqa_dataset: list[DatasetEntry]) -> list[str]:
    instruction = "[I] Na podstawie podanego pytania i kontekstu wygeneruj odpowiedź. [/I]"
    question = f"[Q] {entry.question} [/Q]"

    contexts = [entry.context]

    rand = random.randint(0, 2)
    for _ in range(0, rand):
        context = random.choice(poquad_dataset).context
        contexts.append(context)

    rand = random.randint(0, 2)
    for _ in range(0, rand):
        context = random.choice(polqa_dataset).context
        contexts.append(context)

    shuffled_contexts = random.sample(contexts, len(contexts))

    context = f"[C] {'\n'.join(shuffled_contexts)} [/C]"

    prompts = []

    for answer in entry.answers:
        answer = f"[A] {answer} [/A]"
        prompts.append(f"{instruction}\n{question}\n{context}\n{answer}")

    return prompts

In [20]:
# we'll mix the two datasets
# we'll also mix the data and add from 1 to 10 contexts to each sample to make the model more robust
import json
import random

random_15k_poquad = random.sample(poquad_dataset, 15000)
random_15k_polqa = random.sample(polqa_dataset, 15000)

data = [{ "text": "xdxd"}]

with open("training.jsonl", "w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")

In [2]:
from elasticsearch import Elasticsearch
from qdrant_client import QdrantClient
from cache.cache import Cache
from repository.es_repository import ESRepository


es_client = Elasticsearch(
    hosts=["http://localhost:9200"],
)
cache = Cache()
qdrant_client = QdrantClient(host="localhost", port=6333)

In [3]:
from common.names import DISTANCES, OPENAI_EMBEDDING_MODEL_NAMES
from repository.qdrant_openai_repository import QdrantOpenAIRepository


openai_repo = QdrantOpenAIRepository.get_repository(qdrant_client, OPENAI_EMBEDDING_MODEL_NAMES[0], DISTANCES[0], cache)

Vectorizer with model text-embedding-3-large initialized
Qdrant openai collection text-embedding-3-large-Cosine repository initialized


In [4]:
from dataset.polqa_dataset_getter import PolqaDatasetGetter
from dataset.poquad_dataset_getter import PoquadDatasetGetter

poquad_dataset_getter = PoquadDatasetGetter()
polqa_dataset_getter = PolqaDatasetGetter()

poquad_dataset = poquad_dataset_getter.get_test_dataset()
polqa_dataset = polqa_dataset_getter.get_test_dataset()

In [5]:
first_entry = poquad_dataset[0]

In [6]:
print(first_entry)

DatasetEntry(id=1, passage_id=8e9f992b92ae52ac74c374772515c9181f469aa1f8c60be3acbd9dc921b36180, title=Miszna, context=Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik., dataset=clarin-pl/poquad question=Czym są pisma rabiniczne?, answers=['kompilację poglądów różnych rabinów na określony temat'], metadata={'answer_start': 43})


In [7]:
result = openai_repo.find(first_entry.question, "clarin-pl-poquad-1000")

In [11]:
print(result.passages[0][0])

Passage(id=8e9f992b92ae52ac74c374772515c9181f469aa1f8c60be3acbd9dc921b36180, title=Miszna context=Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik., title=Miszna, start_index=0, dataset=clarin-pl/poquad, dataset_key=clarin-pl-poquad-1000, metadata={'answer_start': 43})


In [4]:
import tiktoken

# Choose the model's encoding. For GPT-3.5 or GPT-4, use 'cl100k_base'
encoding = tiktoken.get_encoding("cl100k_base")

In [13]:
text = "This is an example of counting tokens using tiktoken."

# Encode the text to get the tokens
tokens = encoding.encode(text)

# Count the number of tokens
token_count = 0

for entry in poquad_dataset:
    tokens = encoding.encode(entry.context)
    token_count += len(tokens)

print(token_count)

2013727


In [8]:
print(poquad_dataset[0])

DatasetEntry(id=1, passage_id=8e9f992b92ae52ac74c374772515c9181f469aa1f8c60be3acbd9dc921b36180, title=Miszna, context=Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik., dataset=clarin-pl/poquad question=Czym są pisma rabiniczne?, answers=['kompilację poglądów różnych rabinów na określony temat'], metadata={'answer_start': 43})


In [3]:
result = es_repository.find("trzech lat zaczęła pojawiać", "clarin-pl-poquad-500")

In [4]:
from common.names import RERANKER_MODEL_NAMES
from rerankers.hf_reranker import HFReranker


hf_reranker = HFReranker(RERANKER_MODEL_NAMES[0], cache)

reranked = hf_reranker.rerank(result, 10, "clarin-pl-poquad-500")

Vectorizer with model sdadas/polish-reranker-large-ranknet initialized


In [5]:
print(reranked.passages)

[(<common.passage.Passage object at 0x2cffe9220>, 1.0), (<common.passage.Passage object at 0x2ceb877d0>, 0.06039770444953183), (<common.passage.Passage object at 0x2cfef4980>, 0.054974635580139763), (<common.passage.Passage object at 0x2cff637a0>, 0.05188988598404347), (<common.passage.Passage object at 0x2cff8c710>, 0.030189352723903107), (<common.passage.Passage object at 0x2ce5167b0>, 0.029972132363524976), (<common.passage.Passage object at 0x2ce517320>, 0.01680768521965796), (<common.passage.Passage object at 0x2c91377d0>, 0.016281863795601363), (<common.passage.Passage object at 0x2c91375c0>, 0.013635325838706823), (<common.passage.Passage object at 0x2c91376e0>, 0.0)]


In [6]:
print(result.passages)

[(<common.passage.Passage object at 0x169f6f140>, 1.0), (<common.passage.Passage object at 0x169f6c920>, 0.4424924892411566), (<common.passage.Passage object at 0x1690a28d0>, 0.16352886251699683), (<common.passage.Passage object at 0x116539c70>, 0.10390777903855804), (<common.passage.Passage object at 0x116538740>, 0.10390777903855804), (<common.passage.Passage object at 0x1165385f0>, 0.04750743487495322), (<common.passage.Passage object at 0x116538470>, 0.011301675103537087), (<common.passage.Passage object at 0x169d42de0>, 0.011301675103537087), (<common.passage.Passage object at 0x127ccd0a0>, 0.0), (<common.passage.Passage object at 0x1696286e0>, 0.0)]


In [7]:
print(es_repository.count_relevant_documents("3888", "ipipan-polqa-500"))

0


In [6]:
from qdrant_client import QdrantClient

from common.names import DISTANCES, MODEL_NAMES
from repository.qdrant_repository import QdrantRepository


qdrant_client = QdrantClient(host="localhost", port=6333)
qdrant_repository = QdrantRepository.get_repository(qdrant_client, MODEL_NAMES[0], DISTANCES[0], cache)

Vectorizer with model sdadas/mmlw-retrieval-roberta-large initialized
Qdrant collection sdadas-mmlw-retrieval-roberta-large-Cosine repository initialized


In [7]:
print(qdrant_repository.count_relevant_documents("3888", "ipipan-polqa-500"))

5


In [8]:
result = qdrant_repository.find("trzech lat zaczęła pojawiać", "clarin-pl-poquad-500")

In [9]:
print(result.passages[0][0])

Passage(id=2127, title=Pawie oczko context=przypadały co najmniej dwie lub trzy samice. W przeciwnym wypadku samice będą stale zestresowane i wyczerpane ciągłymi zalotami., title=Pawie oczko, start_index=399, dataset=clarin-pl/poquad, dataset_key=clarin-pl-poquad-500, metadata={'answer_start': 70})


In [10]:
from retrievers.hybrid_retriever import HybridRetriever


hybrid_retriever = HybridRetriever(es_repository, qdrant_repository, "clarin-pl-poquad-500", 0.5)

In [11]:
result = hybrid_retriever.get_relevant_passages(
    "Jakie nowy sprzęt zamontowano w modelu T.8?"
)

In [12]:
print(result.passages[4][0])

Passage(id=695, title=Operacja Rawiw context=aby z ocalałych pojazdów i radaru konstrukcji radzieckiej zabrać wyposażenie i części w celu zbadania ich po powrocie., title=Operacja Rawiw, start_index=1187, dataset=clarin-pl/poquad, dataset_key=clarin-pl-poquad-500, metadata={'answer_start': 74})


In [34]:
from transformers import AutoModelForCausalLM
from optimum.quanto import QuantizedModelForCausalLM, qint4

qmodel = QuantizedModelForCausalLM.from_pretrained("./qra-1b-qint4")
qmodel = qmodel.to("mps")


ValueError: No quantization map found in ./qra-1b-qint4: is this a quantized model ?

In [29]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("OPI-PG/Qra-1b")

In [30]:
input_ids = tokenizer("Warszawa to...", return_tensors="pt").input_ids.to("mps")

In [31]:
import torch

for _ in range(50):  # Generate 50 tokens dynamically
    # Generate the next token
    output = qmodel.generate(input_ids, max_new_tokens=1)

    # Concatenate the new token to the existing input_ids
    input_ids = torch.cat([input_ids, output[:, -1:]], dim=-1).to("mps")

    new_token_id = output[:, -1].tolist()
    new_token = tokenizer.decode(new_token_id, skip_special_tokens=True)

    # Print the new token dynamically
    print(new_token, end="")

1000zł.WynikającyztegokalkulacjicenymieszkaniawWarszawiewynosi10

KeyboardInterrupt: 